In [ ]:
'''
https://www.geeksforgeeks.org/extract-ip-address-from-file-using-python/
'''

In [2]:
#Parse export from vCenter and create IP subnets table

import pprint as pp
import re
import ipaddress
#from mac_vendor_lookup import MacLookup

In [46]:
#Clean duplicate field from comma separated string
def cleandup(ip):
    iplist = ip.split(',')
    #return ','.join(list(dict.fromkeys(iplist)))
    return ','.join(list(set(iplist)))

In [47]:
#Return range of IP addressess for particular subnet
#takes first IP address if there are more IP's separated by coma
def getrange(ip, mask):
    try:
        ip = ip.split(',', 1)[0].strip()
        subnet = ipaddress.ip_network((ip, mask), strict = False)
        return str(subnet.network_address) + ' - ' +  str(subnet.broadcast_address)
    except (ValueError, TypeError):
        return('NO VALID IP')

In [56]:
#Change network format from XXX.YYY.ZZZ.NNN to the short notation with /XX 
def shortnet(ip, mask):
    try:
        ip = ip.split(',', 1)[0].strip()
        subnet = ipaddress.ip_network((ip, mask), strict = False)
        return str(subnet)
    except (ValueError, TypeError):
        return('NO VALID NETWORK')

In [65]:
fields = {'VM':None, 'Host':None, 'OS':None,
          'Tools':None, 'NicType':None, 'VLAN':None,
          'IP':None, 'Gateway':None, 'Subnet Mask':None,
          'Network':None, 'IP range':None,
          'DNS':None,'MAC':None, 'File': None}

header = ['VM', 'Host', 'OS', 'Tools', 'NicType', 'VLAN',
          'IP', 'Gateway', 'Subnet Mask', 'Network', 'IP range', 'DNS','MAC', 'File']

In [71]:
#tags to mark start and end block
starttag = 'VM'
endtag = 'MAC'
separator = '|'

#set to true if looping inside start/end block
inside = False

filename = 'EE_VMs_Network.txt'
output = filename + '.csv'

with open(filename, encoding='utf8') as file, open(output, 'w', encoding='utf8') as output:
    #write csv header
    output.write(separator.join(header))
    output.write('\n')
    
    for row in file:
        #inside marker if row starts with start tag
        if row.startswith(starttag):
            inside = True
            
        #asign values to the keys while we are inside start-end block
        if inside:
            row_list = row.split(':', 1)
            
            #check if split have 2 fields, if only one- field continues on the next line
            if len(row_list) == 2:
                key = row_list[0].strip()
                value = row_list[1].strip()
                fields[key] = value
            #append rest of next line to the previous key value
            else:
                fields[key] = fields[key] + row.strip()
            
            #inside/outside status if endtag is hit
            if row.startswith(endtag):
                #Add calculated fields
                fields['IP range'] = getrange(fields['IP'], fields['Subnet Mask'])
                fields['Network'] = shortnet(fields['IP'], fields['Subnet Mask'])
                fields['File'] = filename
                
                #Clean duplicates inside fields and write to file
                csv_string = ''
                for m in header:
                    csv_string = csv_string + cleandup(fields[m]).strip() + separator
                csv_string = csv_string + '\n'
                output.write(csv_string)
                #Set status to indicate we are outside start/end tags
                inside = False

In [2]:
#Create summary subnets when given first and last IP address
def summarize_ip(ipfile):
    iplist = []
    with open(ipfile, encoding='utf8') as f:
        for line in f:
            iplist.append(ipaddress.ip_address(line.strip()))
        #remove duplicates
        iplist = list(dict.fromkeys(iplist))
            
    ip_min = min(iplist)
    ip_max = max(iplist)
    
    print('MIN:', ip_min, 'MAX:', ip_max, '\n')
    for m in sorted(iplist):
        print(m)
    print()
    
    ip_range = ipaddress.summarize_address_range(ip_min, ip_max)
    for i in ip_range:
        #print(i, list(i.hosts()))
        print(i, ',', i.network_address, '-', i.broadcast_address)

In [11]:
#Read file with IP addressess and migration status (comma separated) and return dictionary
def read_ip(ipfile):
    iplist = {}
    with open(ipfile, encoding='utf8') as f:
        for line in f:
            pair = line.split('\t')
            key = pair[0].strip()
            value = pair[1].strip()
            iplist[key]= value
            #iplist.append(ipaddress.ip_address(line.strip()))
    return iplist

#take list of IP addressess and fit them into predefines net divided into subnets (offset)
iplist = read_ip('10.5.64.0.txt')
ipnetwork = '10.5.64.0/24'
offset = 2

subnets = list(ipaddress.ip_network(ipnetwork, strict = False).subnets(offset))
for net in subnets:
    
    network = net.network_address
    broadcast = net.broadcast_address
    
    print(net, str(network), '-', str(broadcast), '\n')
    for ip_s in iplist.keys():
        ip = ipaddress.ip_address(ip_s)
        if ip in net:
            #check if IP address is same as broadcast or subnet
            if ip == network or ip == broadcast:
                warn = '*'
            else:
                warn = ''
            print(ip_s, iplist[ip_s], warn)
    print()

10.5.64.0/26 10.5.64.0 - 10.5.64.63 

10.5.64.26 Yes 
10.5.64.6 Yes 
10.5.64.59 No 
10.5.64.25 No 
10.5.64.32 No 

10.5.64.64/26 10.5.64.64 - 10.5.64.127 


10.5.64.128/26 10.5.64.128 - 10.5.64.191 


10.5.64.192/26 10.5.64.192 - 10.5.64.255 




In [6]:
summarize_ip('Baltics sorted IP list.txt')

MIN: 10.2.42.244 MAX: 212.107.40.66 

10.2.42.244
10.5.25.10
10.5.25.24
10.5.25.25
10.5.25.29
10.5.25.45
10.5.25.50
10.5.25.53
10.5.25.54
10.5.25.102
10.5.25.169
10.5.25.170
10.5.25.252
10.5.38.10
10.5.64.2
10.5.64.4
10.5.64.6
10.5.64.7
10.5.64.9
10.5.64.15
10.5.64.25
10.5.64.26
10.5.64.32
10.5.64.45
10.5.64.46
10.5.65.77
10.5.65.86
10.5.65.87
10.5.65.100
10.5.65.110
10.5.65.111
10.5.65.123
10.5.65.124
10.5.65.125
10.5.65.128
10.5.65.179
10.5.65.184
10.5.65.185
10.5.65.197
10.5.65.216
10.5.65.222
10.5.65.225
10.5.65.227
10.5.65.229
10.5.65.240
10.5.88.77
10.5.88.80
10.5.88.81
10.5.88.82
10.5.88.91
10.5.88.99
10.5.88.102
10.5.96.19
10.5.96.25
10.5.96.26
10.5.96.34
10.5.96.35
10.5.96.36
10.5.96.37
10.5.96.44
10.5.96.47
10.5.96.76
10.5.96.77
10.5.96.82
10.5.96.83
10.5.96.85
10.5.96.87
10.5.96.88
10.5.96.91
10.5.96.93
10.5.96.94
10.5.96.95
10.5.96.97
10.5.96.99
10.5.96.100
10.5.96.101
10.5.96.102
10.5.96.105
10.5.96.110
10.5.96.111
10.5.96.112
10.5.96.113
10.5.96.114
10.5.96.115
10.5.96.11